# Unificação dos dados

## Obtenção e carregamento de dados  

In [ ]:
#importação de bibliotecas
import pandas as pd

In [ ]:
#Carregamento de dados limpos da pasta ../data/
df_alunos = pd.read_csv('../Data/tab_Alunos.csv')
df_acessos = pd.read_csv('../Data/tab_Acessos.csv')
df_cursos = pd.read_csv('../Data/tab_Cursos_Oferta.csv')
df_Motivo = pd.read_csv('../Data/tab_Motivo.csv')
df_Motivo_Classificacao = pd.read_csv('../Data/tab_Motivo_Classificacao.csv')
df_Professores = pd.read_csv('../Data/tab_Professores.csv')

### Alunos

In [ ]:
df_alunos.head()

### Acessos

In [ ]:
df_acessos.head()

### Cursos

In [ ]:
df_cursos.head()

### Motivo

In [ ]:
df_Motivo.head()

### Motivo Classificação

In [ ]:
df_Motivo_Classificacao.head() 

### Professores

In [ ]:
df_Professores.head()

## Preparação dos dados para união

### Acessos

Verifica o formato dos dados e a presença de valores nulos.

In [ ]:
df_acessos.info()

Ajusta o formato dos dados

In [ ]:
#Define a coluna dt_acesso como datetime
df_acessos['dt_acesso'] = pd.to_datetime(df_acessos['dt_acesso'])

df_acessos.info()

Transforma o dataframe de acesso em um conjunto de dados agrupados por aluno, com as colunas referentes as informações de acesso de cada aluno

In [ ]:
df_acessos = df_acessos.groupby('cd_aluno').agg(
    primeiro_login=('dt_acesso', 'min'),
    ultimo_login=('dt_acesso', 'max'),
    total_logins=('dt_acesso', 'count'),
).assign(
    dias_desde_primeiro_login=lambda df: (pd.to_datetime('2023-12-31') - df['primeiro_login']).dt.days,
    media_acessos_por_dia=lambda df: df['total_logins'] / df['dias_desde_primeiro_login']
)

df_alunos.rename(columns={'ds_curso': 'curso'}, inplace=True)

df_acessos.head()


### Professores

Verifica o formato dos dados e a presença de valores nulos.

In [ ]:
#Exibe as informações do dataframe
df_Professores.info()

Transforma o dataframe de professores em um conjunto de dados agrupados por curso e semestre_que_leciona, com as colunas referentes as informações de cada professor

In [ ]:
# renomeia coluna para facilitar agrupamento com tabela de alunos
df_Professores.rename(columns={'semestre_que_leciona': 'semestre_atual'}, inplace=True)

# Agrupar por cd_curso e semestre_que_leciona
df_Professores = df_Professores.groupby(['cd_curso', 'semestre_atual']).agg(
    qtd_professores=('cd_professor', 'nunique'),
    qtd_professores_mestrado=('nivel_formacao_professor', lambda x: (x == 'Mestrado').sum()),
    qtd_professores_doutorado=('nivel_formacao_professor', lambda x: (x == 'Doutorado').sum()),
    qtd_professores_especialista=('nivel_formacao_professor', lambda x: (x == 'Especialização').sum()),
    qtd_professores_graduado=('nivel_formacao_professor', lambda x: (x == 'Graduação').sum()),
    qtd_professores_M=('genero', lambda x: (x == 'M').sum()),
    qtd_professores_F=('genero', lambda x: (x == 'F').sum()),
    idade_min_professores=('idade', 'min'),
    idade_max_professores=('idade', 'max'),
    idade_media_professores=('idade', 'mean')
).reset_index()

#Exibe as informações do dataframe
df_Professores.info()

# Exibir as estatísticas
df_Professores.head()



## União dos dados

### Alunos + Acessos

In [ ]:
#Une os dataframes df_alunos e df_acessos
df_dataset = pd.merge(df_alunos, df_acessos, on='cd_aluno', how='left')
#Exibe as informações do dataframe
df_dataset.info()
#Exibe as primeiras linhas do dataframe
df_dataset.head()

In [ ]:
# Preenche os valores nulos nas colunas 'primeiro_login' e 'ultimo_login' com a data do último dia de 2023
df_dataset['primeiro_login'] = df_dataset['primeiro_login'].fillna(pd.Timestamp('2023-12-31'))
df_dataset['ultimo_login'] = df_dataset['ultimo_login'].fillna(pd.Timestamp('2023-12-31'))

# Preenche os valores nulos nas colunas 'dias_desde_primeiro_login' e 'media_acessos_por_dia' com 0
df_dataset['total_logins'] = df_dataset['total_logins'].fillna(0).astype('int64')
df_dataset['dias_desde_primeiro_login'] = df_dataset['dias_desde_primeiro_login'].fillna(0).astype('int64')
df_dataset['media_acessos_por_dia'] = df_dataset['media_acessos_por_dia'].fillna(0).astype('float64')

#Exibe as informações do dataframe
df_dataset.info()

#Exibe as primeiras linhas do dataframe
df_dataset.head()

### Alunos + Acessos + Cursos

In [ ]:
#Une os dataframes df_dataset e df_Cursos
df_dataset = pd.merge(df_dataset, df_cursos[['cd_curso','dt_inauguracao', 'semestres_existencia']], on='cd_curso', how='left')

#Renomeia as colunas dt_inauguracao e semestres_existencia para dt_inauguracao_curso e semestres_existencia_curso
df_dataset.rename(columns={'dt_inauguracao': 'dt_inauguracao_curso', 'semestres_existencia': 'semestres_existencia_curso'}, inplace=True)

df_dataset.head()

### Alunos + Acessos + Cursos + Motivo

#### União auxiliar - Motivos e Motivos Classificação

Une as tabelas auxiliares de motivo e motivo_classificacao

In [ ]:
#Une os dataframes df_Motivo e df_Motivo_Classificacao
df_dataset_motivo = df_Motivo.merge(df_Motivo_Classificacao[['ds_motivo', 'categoria_motivo_cancelamento']], how='inner', on='ds_motivo')

#Removida a coluna ds_motivo pois não usada na analise
df_dataset_motivo.drop('ds_motivo', axis=1, inplace=True)

#Exibe as informações do dataframe
df_dataset_motivo.info()

#Exibe as primeiras linhas do dataframe
df_dataset_motivo.head()

Ajusta o tipo de dados

In [ ]:
#Ajusta o tipo de dado da coluna dt_saida para datetime
df_dataset_motivo['dt_saida'] = pd.to_datetime(df_dataset_motivo['dt_saida'])

#Exibe as informações do dataframe
df_dataset_motivo.info()

#Exibe as primeiras linhas do dataframe
df_dataset_motivo.head()

Verifica se existe duplicidade de dados (Erros lançados pelo sistema de origem da IES)

In [ ]:
#Identificacao de duplicidades 
duplicados = df_dataset_motivo.duplicated(['cd_aluno','cd_curso']).sum()
print("Registros duplicados:" , duplicados)

Removendo duplicidades

In [ ]:
# Encontra o índice da data de saída mais recente para cada aluno
idx = df_dataset_motivo.groupby(['cd_aluno','cd_curso'])['dt_saida'].idxmax()

# Use esses índices para selecionar as linhas correspondentes do dataframe
df_dataset_motivo = df_dataset_motivo.loc[idx]

#Identificacao de duplicidades
duplicados = df_dataset_motivo.duplicated(['cd_aluno','cd_curso']).sum()
print("Registros duplicados:" , duplicados)

Exibição dos dados

In [ ]:
#Exibe as informações do dataframe
df_dataset_motivo.info()

#Exibe as primeiras linhas do dataframe
df_dataset_motivo.head()

#### União final

In [ ]:
#Une os dataframes df_dataset e df_dataset_motivo
df_dataset = pd.merge(df_dataset, df_dataset_motivo[['cd_aluno', 'cd_curso','categoria_motivo_cancelamento']], how='left', on=['cd_aluno','cd_curso'])

#Exibe as informações do dataframe
df_dataset.info()

Tratar valores nulos após agrupamento

In [ ]:
#preenche com o texto "Abandono" os valores nulos da coluna 'categoria_motivo_cancelamento'
df_dataset['categoria_motivo_cancelamento'] = df_dataset['categoria_motivo_cancelamento'].fillna('Abandono').where(df_dataset['evasao'] == 1, 'NA')

#Exibe as informações do dataframe
df_dataset.info()

#Exibe as primeiras linhas do dataframe
df_dataset.head()

### Alunos + Acessos + Cursos + Motivo + Professores

Une os dataframes df_dataset e df_professores

In [ ]:
#Une os dataframes df_dataset e df_Professores
df_dataset = pd.merge(df_dataset, df_Professores, how='left', on=['cd_curso','semestre_atual'])

#Exibe as informações do dataframe
df_dataset.info()

Verifica registros com dados nulos

In [ ]:
#Exibe os registros que possuem linhas com valores nulos
df_dataset[df_dataset.isna().any(axis=1)]

Remove registros com dados nulos

In [ ]:
#Removidos alunos que não tinham nenhuma amarração com professor
df_dataset = df_dataset.dropna()

Exibe as informações

In [ ]:
#Exibe as informações do dataframe
df_dataset.info()

#Exibe as primeiras linhas do dataframe
df_dataset.head()

## Verificação e Exportação de Dados

### Verificação

In [ ]:
#Exibir informações sobre o dataframe
df_dataset.info()

#Exibir estatísticas descritivas do dataframe
df_dataset.describe()

### Exportação

In [ ]:
#Exporta o dataframe para um arquivo CSV
df_dataset.to_csv('../data/dataset.csv', index=False)